# Get Stock Price Data

Steps to get the script working for you or just use the data in dat folder

This works only on Mac and LINUX and I am not sure how this might translate in MSDOS systems such as Windows

* go online to: https://rapidapi.com/
* Register and get an API key
* either put this in theis script directly OR
> * Create a file in your home names "~/.keys/keys.json" with the key 
        {
            "AV_API_KEY": "YOUR API KEY HERE" ,
            "NEWSAPI_KEY": "DONT WORRY ABOUT THIS FOR NOW",
        }

This create files under your data/stock_price and a combined file named "stockdata.csv"
You may use that file directly for your assignments


This will pull prices for all the symbols stored in symbols.txt



In [107]:
%%writefile  getstocksdata.py
#!/usr/local/bin/python 

#GENERATED BY COMP4449/capstone/stock_prediction1/01_GetData.ipynb

import os, datetime, glob, sys, time, datetime, colabexts, shutil, argparse
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from colabexts.jcommon import *

#-----------------------------------------------------------------------------------
def getkey(key='password'):
    API_KEY, lines, file =None, None, os.path.expanduser('~/.keys/keys.json')
    if os.path.exists(file):
        with open(file, 'r') as f:
            r = f.read()
        j = eval(r)
        return j['AV_API_KEY'], j['NEWSAPI_KEY']
    else:
        raise Exception ("Please supply API_KEY")

    return avk, newsk;
        

#-----------------------------------------------------------------------------------
'''
This will read required symbols and saves them to data directory
'''
def save_data(symbol, API_KEY="", check=True, savein="data/"):
    from alpha_vantage.timeseries import TimeSeries
    if (savein and not savein.endswith("/")):
        savein = savein + "/"
    outf = f'{savein}daily_{symbol}.csv'
    if (check and os.path.exists(outf)):
        dt = datetime.datetime.fromtimestamp(os.path.getmtime(outf))
        dn = datetime.datetime.now()
        ts = (dn - dt)
        hr = (ts.days * 24 * 60 * 60 + ts.seconds)//60//60
        if (hr < 8): #if it was created less than 4 hours ago
            print(f"{outf:22} exists, ... crested less than 8 hours!! at {dt} ")
            return;
    ts = TimeSeries(key=API_KEY, output_format='pandas')
    
    retry = 0
    data = None
    while retry <= 5:
        try:
            print(f"Getting data for {symbol}")
            data, meta_data = ts.get_daily(symbol, outputsize='full')
            break;
        except ValueError as ve:
            retry += 1
            print(f"Sleep for a Minute, {retry}/5 attempts");
            time.sleep(60)
            
    if data is None:
        print(f"Could not get data for {symbol}")
        return data
    
    data.insert(0, 'timestamp', value=data.index)

    data.columns = 'timestamp,open,high,low,close,volume'.split(',')
    print( f"saving to output: {outf}" )
    data.to_csv(outf, index=False)
    return data

'''
Read all the files in data with daily_*, reads them and returns a list
'''
def read_data(dirname="", pattern="daily_*"):
    a={}
    for f in glob.glob( f'{dirname}/{pattern}'):
        symbol = os.path.basename(os.path.splitext(f)[0]).split("_")[1]
        print(f'Reading {f} Symbol: {symbol}')
        df = pd.read_csv(f)
        df.sort_values(by='timestamp', ascending=False, inplace=True)
        df.index=(range(0,len(df)))
        ncs = ['timestamp'] + [f'{symbol}_{c}' for c in df.columns[1:]]
        df.columns = ncs
        a[symbol] = df
        
    minrows = min([len(d) for d in a.values()])
    return a, minrows, df

'''
Combines all the dataframes into one

The problem for us to join multiple index funds from ASIA is that they have different holidays
Therefore we have gaps in the trading days. Therefore 
'''
def combine_data(a):
    # Different excahnges have various holidays, therefore values may be missing  
    # Get All Data Frames and their corresponding time stamps
    #
    ar=np.array([d['timestamp'].values for d in a.values()])
    at=np.concatenate(ar)
    at=set(at)
    af = pd.DataFrame()
    af['timestamp'] = list(at);

    for k,v in a.items():
        #print(f"Getting {k:32} \r", end='')
        af=pd.merge(af,v, how="left", left_on="timestamp".split(), right_on="timestamp".split())
    print()
    af.sort_values(by='timestamp', ascending=True, inplace=True)
    af.dropna(inplace=True)
    af = af.reset_index(drop=True)
    #af = af.fillna(method='ffill' ).fillna(method='bfill')
    return af

#-----------------------------------------------------------------------------------
def getCombined(dirname="", output="stockdata.csv"):
    if (not output):
        print ( "No output specified, Nothing to do!!!")
        
    #Get All data together
    a, maxrows, ldf  = read_data(dirname)
    af = combine_data(a)
    
    if (os.path.exists(output)):
        print(f"{output} exists; moving it to backup {output}.BAK")
        shutil.copyfile( output, output+".BAK")

    dname = os.path.dirname(output)
    if dname and not os.path.exists(dname):
        print("Making directories: ", dname)
        os.makedirs(os.path.dirname(stockfile))
            
    print ( f"Saving top output {output}!!!")
    af.to_csv(output, index=False)
    return af    

#-----------------------------------------------------------------------------------
def getdata(symbols='MSFT GLD GOOGL SPX AAPL IBM', savein="" ):
    API_KEY = None  # Put your API KEY if you need to test data download or just use the data
    API_KEY, NEWS_API_KEY = API_KEY or getkey()

    for f in symbols.split():
        if ( not f or f.startswith ('#')):
            print( f'ignoring {f}')
            continue; # empty lines
        kv = f.split(':')
        k = kv[0]
        v = kv[1] if len(kv) > 1 else k 
        print(f'getting data for {k} => symbol {v}')
        save_data(k, API_KEY=API_KEY, savein=savein)

#-----------------------------------------------------------------------------------
sysargs = None
ASIA    = '6758.T:TOKYO  6501.T:HITACHI 0168.HK:HNGKNG 601288.SS:SHANGAI  SHNZEN'
ASIA    = '601288.SS:SHANGAI'
defsyms = f"MSFT GLD GOOGL AAPL IBM {ASIA}"
deffile = "symbols.txt"
defoutp = "data/stock_price/stockdata.csv"
def addargs():
    global sysargs
    p = argparse.ArgumentParser(f"{os.path.basename(sys.argv[0])}:")
    p.add_argument('-s', '--symbols', type=str, default=defsyms, help=f"Symbols separated ex: '{defsyms}'")
    p.add_argument('-f', '--file'   , type=str, default=deffile, help=f"Symbols from file ex: '{deffile}'")
    p.add_argument('-o', '--output' , type=str, default=defoutp, help= f"o/p def: {defoutp}")
#    p.add_argument('args', nargs=argparse.REMAINDER)
#    p.add_argument('input_files',action="store", type=str, nargs='+', help="input file(s)")

    try:
        sysargs, unknown=p.parse_known_args(sys.argv[1:])
    except argparse.ArgumentError as exc:
        print(exc.message )
        
    if (unknown):
        print("Unknown options: ", unknown)
        #p.print_help()
    return sysargs
#-----------------------------------------------------------------------------------
if __name__ == '__main__':
    if (not inJupyter()):
        t1      = datetime.datetime.now()
        sysargs = addargs()
        symbs   = sysargs.symbols
        if ( os.path.exists(sysargs.file)):
            symbs += "\n" + colabexts.jcommon.readFile(sysargs.file)
           
        dname   = os.path.dirname(sysargs.output) +"/"
        getdata(symbs, savein=dname)
        #getCombined(dname, sysargs.output);
        t2 = datetime.datetime.now()
        print(f"All Done in {str(t2-t1)} ***")
    else:
        pass
        '''
        a, minrows, ldf  = read_data()
        stockfile="data/stockdata.csv"
        af= combine_data(a, stockfile)
        '''

Overwriting getstocksdata.py
